In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import torchvision.models as models
from timm.data.loader import MultiEpochsDataLoader

from model import *
from data_loader import LoadDataset

In [2]:
data_cfgs = {"name": "DL20", "num_classes": 20, "dir":"./data/DL20"}
train_cfgs = {"batch_size": 128, "lr": 0.0001}

### load small version of ResNet
model = Small_ResNet(BasicBlock, [3, 3, 3], num_classes=data_cfgs['num_classes']).to('cuda')

### load train/valid/test dataset
train_dataset = LoadDataset(data_cfgs["dir"], mode="train", random_flip=True)
valid_dataset = LoadDataset(data_cfgs["dir"], mode="valid", random_flip=False)
# test_dataset = LoadDataset(data_cfgs["dir"], mode="test", random_flip=False)

### warp dataset using dataloader
train_dataloader = MultiEpochsDataLoader(train_dataset, batch_size=train_cfgs["batch_size"], shuffle=True, pin_memory=True, drop_last=True, num_workers=64)
valid_dataloader = MultiEpochsDataLoader(valid_dataset, batch_size=train_cfgs["batch_size"], shuffle=False, pin_memory=True, drop_last=False, num_workers=64)
# test_dataloader = MultiEpochsDataLoader(test_dataset, batch_size=train_cfgs["batch_size"], shuffle=False, pin_memory=True, drop_last=False)

### define Adam optimizer: one of the popular optimizers in Deep Learning community
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), train_cfgs["lr"], eps=1e-6)

### define cross-entropy loss for classification
criterion = nn.CrossEntropyLoss()


In [3]:
def train_model(model, patience, num_epochs):

    train_losses = []
    valid_losses = []
    mean_train_losses = []
    mean_valid_losses = [] 
    p = 0
    min_valid_loss = 100

    for epoch in range(1, num_epochs + 1):

        model.train() 
        for train_data, target in train_dataloader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                train_data, target = train_data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(train_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        model.eval() 
        for valid_data, target in valid_dataloader:
            if torch.cuda.is_available():
                device = torch.device("cuda")
                valid_data, target = valid_data.to(device), target.to(device)
            output = model(valid_data)
            loss = criterion(output, target)
            valid_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        valid_loss = np.mean(valid_losses)
        mean_train_losses.append(train_loss)
        mean_valid_losses.append(valid_loss)
        
        if min_valid_loss > valid_loss:
            min_valid_loss = valid_loss
            # print(f'min_valid_loss: {min_valid_loss}')

        epoch_len = len(str(num_epochs))
        print_msg = (f'[{epoch:>{epoch_len}}/{num_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        print(print_msg)
        
        train_losses = []
        valid_losses = []
        
        if min_valid_loss < valid_loss and epoch > 1:
            p = p + 1
            #print(f'patience: {p}')
        else:
            p = 0
            torch.save(model.state_dict(), 'bestmodel.pt')
            print("Saving Model...")
        
        if patience == p:
            print("Early stopping")
            break

    model.load_state_dict(torch.load('bestmodel.pt'))

    return  model, mean_train_losses, mean_valid_losses

In [4]:
def weight_reset(m):
    if isinstance(m, nn.Linear):
        m.reset_parameters()

def print_test_accuracy(model, test_loader):
  class_correct = list(0. for i in range(20))
  class_total = list(0. for i in range(20))

  model.eval()

  with torch.no_grad():
    for test_data, target in test_loader:
        if torch.cuda.is_available():
            device = torch.device("cuda")
            test_data, target = test_data.to(device), target.to(device)
        output = model(test_data)
        _, predicted = torch.max(output, 1)
        c = np.squeeze(predicted.eq(target.data.view_as(predicted)))
        for i in range(len(target.data)):
            label = target.data[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

  model.apply(weight_reset)

  print('Test Accuracy (Total): %.2f %% (%d/%d)\n' % (
      100. * np.sum(class_correct) / np.sum(class_total),
      np.sum(class_correct), np.sum(class_total)))

In [5]:
num_epochs = 1000
patience = 15

final_model, train_losses, valid_losses= train_model(model, patience, num_epochs)
print_test_accuracy(final_model, valid_dataloader)

[   1/1000] train_loss: 3.00670 valid_loss: 2.92339
Saving Model...
[   2/1000] train_loss: 2.82268 valid_loss: 2.75782
Saving Model...
[   3/1000] train_loss: 2.70124 valid_loss: 2.65968
Saving Model...
[   4/1000] train_loss: 2.61736 valid_loss: 2.59399
Saving Model...
[   5/1000] train_loss: 2.54196 valid_loss: 2.55143
Saving Model...
[   6/1000] train_loss: 2.47795 valid_loss: 2.52382
Saving Model...
[   7/1000] train_loss: 2.42264 valid_loss: 2.49832
Saving Model...
[   8/1000] train_loss: 2.38474 valid_loss: 2.47033
Saving Model...
[   9/1000] train_loss: 2.34093 valid_loss: 2.41841
Saving Model...
[  10/1000] train_loss: 2.30655 valid_loss: 2.42585
[  11/1000] train_loss: 2.27170 valid_loss: 2.34237
Saving Model...
[  12/1000] train_loss: 2.23674 valid_loss: 2.34554
[  13/1000] train_loss: 2.21481 valid_loss: 2.31239
Saving Model...
[  14/1000] train_loss: 2.18919 valid_loss: 2.30041
Saving Model...
[  15/1000] train_loss: 2.15754 valid_loss: 2.27747
Saving Model...
[  16/1000] 